In [1]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from cassandra.cluster import Cluster

In [2]:
cassandra_host = ['127.0.0.1']
cassandra_port = 9042
cassandra_keyspace = 'assignment'
file_path = '/home/ishan/Desktop/Assignment/dataset/per-vehicle-records-2020-01-31.csv'

In [3]:
conf = (SparkConf()
        .setAppName("Assignment")
        .set('spark.cassandra.connection.host', cassandra_host[0])
        .set('spark.cassandra.connection.port', cassandra_port)
        .set('spark.jars', '/home/ishan/Desktop/Assignment/spark-cassandra-connector_2.11-2.0.12.jar,/home/ishan/Desktop/Assignment/jsr166e-1.1.0.jar')
        .setMaster('local[*]'))
sc = SparkContext.getOrCreate(conf = conf)
sqlContext = SQLContext(sc)

In [4]:
cassandra_create_queries = [
    "CREATE KEYSPACE IF NOT EXISTS " + cassandra_keyspace + " WITH replication = {'class':'SimpleStrategy', 'replication_factor':1};",
    'use ' + cassandra_keyspace + ';',
    'create table if not exists q1 (classname text primary key, percent double);',
    'create table if not exists q2_max (hour int primary key, count int);',
    'create table if not exists q2_min (hour int primary key, count int);',
    'create table if not exists q3_morning (hour int primary key, count int);',
    'create table if not exists q3_evening (hour int primary key, count int);',
    'create table if not exists q4 (lanename text primary key, avg_speed double);',
    'create table if not exists q5 (lanename text primary key, avg_cosit double, count int);'
]

def setupCassandra():
    cluster = Cluster(cassandra_host)
    session = cluster.connect()
    for query in cassandra_create_queries:
        print("Executing - {}".format(query))
        session.execute(query)

In [5]:
def saveToCassandra(df, table_name):
    (df
     .write
     .format("org.apache.spark.sql.cassandra")
     .mode('append')
     .options(table=table_name, keyspace=cassandra_keyspace)
     .save())

In [6]:
setupCassandra()

/home/ishan/.local/lib/python3.5/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  


Executing - CREATE KEYSPACE IF NOT EXISTS assignment WITH replication = {'class':'SimpleStrategy', 'replication_factor':1};
Executing - use assignment;
Executing - create table if not exists q1 (classname text primary key, percent double);
Executing - create table if not exists q2_max (hour int primary key, count int);
Executing - create table if not exists q2_min (hour int primary key, count int);
Executing - create table if not exists q3_morning (hour int primary key, count int);
Executing - create table if not exists q3_evening (hour int primary key, count int);
Executing - create table if not exists q4 (lanename text primary key, avg_speed double);
Executing - create table if not exists q5 (lanename text primary key, avg_cosit double, count int);


In [7]:
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(file_path)

In [8]:
# 1. Calculate the usage of Irish road network in terms of percentage grouped by vehicle category.

In [9]:
total_vehicles = df.count()

q1DF = (df
        .groupBy('classname')
        .count()
        .withColumn('percent', (col('count') * 100) / total_vehicles)
        .select('classname', 'percent')
        .orderBy(desc('percent')))

q1DF.show()

saveToCassandra(q1DF.filter(col("classname").isNotNull()), "q1")

+---------+--------------------+
|classname|             percent|
+---------+--------------------+
|      CAR|   80.25858594040197|
|      LGV|  11.194464465420944|
|  HGV_ART|   4.397450167807071|
|  HGV_RIG|  2.7310861887745705|
|      BUS|  0.6871114761643508|
|  CARAVAN| 0.42912036442325563|
|    MBIKE| 0.29486205142905475|
|     null|0.007319345578788326|
+---------+--------------------+



In [10]:
# 2. Calculate the highest and lowest hourly flows on M50 - show the hours and total number of vehicle counts.

In [11]:
hourDF = df.groupBy('hour').count()

maxDF = hourDF.orderBy(desc('count')).limit(1)
minDF = hourDF.orderBy('count').limit(1)

maxDF.show()
minDF.show()

saveToCassandra(maxDF, "q2_max")
saveToCassandra(minDF, "q2_min")

+----+------+
|hour| count|
+----+------+
|  16|385850|
+----+------+

+----+-----+
|hour|count|
+----+-----+
|   2|13682|
+----+-----+



In [12]:
# 3. Calculate the evening and morning rush hours on M50 - show the hours and the total counts.

In [13]:
hourDF = df.groupBy('hour').count().orderBy('hour')

morningDF = hourDF.filter((col('hour') < 12) & (col('hour') >= 4))
eveningDF = hourDF.filter((col('hour') < 20) & (col('hour') >= 16))

morningDF.show()
eveningDF.show()

saveToCassandra(morningDF, "q3_morning")
saveToCassandra(eveningDF, "q3_evening")

+----+------+
|hour| count|
+----+------+
|   4| 27187|
|   5| 61937|
|   6|198369|
|   7|299784|
|   8|352862|
|   9|277509|
|  10|256183|
|  11|246847|
+----+------+

+----+------+
|hour| count|
+----+------+
|  16|385850|
|  17|367269|
|  18|314085|
|  19|232409|
+----+------+



In [14]:
# 4. Calculate average speed between each junction on M50 (e.g., junction 1 - junction 2, junction 2 - junction 3, etc.).

In [15]:
q4DF = (df
        .groupBy('lanename')
        .agg(mean('speed').alias("avg_speed"))
        .orderBy(desc('avg_speed')))

q4DF.show()

saveToCassandra(q4DF, "q4")

+--------------------+------------------+
|            lanename|         avg_speed|
+--------------------+------------------+
| Southbound 1 (slow)| 135.4469130170314|
|       Northbound 2 |122.31002458344715|
|        Eastbound  2|114.68716172331673|
|  Eastbound 2 (fast)|113.59000942507083|
| Southbound 2 (fast)|111.72458022387893|
|  Westbound 2 (fast)|111.34068965517257|
| Northbound 2 (fast)|110.21109738884894|
|          southbound| 104.4090909090909|
|         Nortbound 1|103.95987028779895|
| Northbound 1 (slow)|103.63843987902779|
|  Westbound 3 (fast)|103.47554310278697|
|  Eastbound 3 (fast)|100.55896097639352|
|        Southbound 2| 97.79121728990314|
|        Northbound 2| 97.76209841746629|
|  Westbound 2 (slow)| 95.40281196241926|
|Southbound Mainli...| 95.25522388059701|
|         Westbound 2| 93.36724880445983|
|         Eastbound 2| 92.79648071706569|
|   Eastbound on slip|  92.7741935483871|
|        Southbound 1| 92.74038016587762|
+--------------------+------------

In [16]:
# 5. Calculate the top 10 locations with highest number of counts of HGVs (class). Map the COSITs with their names given on the map.

In [17]:
q5DF = (df
        .filter(col('classname').contains('HGV'))
        .groupBy('lanename')
        .agg(mean('cosit').alias('avg_cosit'), count('lanename').alias('count'))
        .orderBy(desc('count')))

q5DF.show(10)

saveToCassandra(q5DF, "q5")

+------------+------------------+-----+
|    lanename|         avg_cosit|count|
+------------+------------------+-----+
|Northbound 1|20693.790824685962|47606|
|Southbound 1| 20370.47938361651|47438|
| Westbound 1| 47984.26970280579|26481|
| Eastbound 1|50842.564949674364|25335|
|  Northbound|11535.248807024242|20956|
|  Southbound| 15721.02364244845|18526|
|Northbound 2| 6737.779673675744|17406|
|Southbound 2| 5250.837445297139|15767|
|   Eastbound|11569.320689406504|13867|
|   Westbound|11157.380619527628|13591|
+------------+------------------+-----+
only showing top 10 rows

